# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 1:22PM,264672,10,MSP222665-Hold_Rx(AL),Methapharm-G,Released
1,Jun 13 2023 12:58PM,264671,19,N18121,"NAPP Technologies, LLC",Released
2,Jun 13 2023 12:45PM,264669,10,CTF0011788,"Citieffe, Inc.",Released
3,Jun 13 2023 12:45PM,264669,10,CTF0011789,"Citieffe, Inc.",Released
4,Jun 13 2023 12:41PM,264667,10,ALK39067,Alikai Health,Released
5,Jun 13 2023 12:41PM,264667,10,ALK39332,Alikai Health,Released
6,Jun 13 2023 12:41PM,264667,10,ALK39363,Alikai Health,Executing
7,Jun 13 2023 12:41PM,264667,10,ALK39349,Alikai Health,Executing
8,Jun 13 2023 12:41PM,264667,10,ALK39373,Alikai Health,Executing
9,Jun 13 2023 12:40PM,264666,19,ADV80020966,"AdvaGen Pharma, Ltd",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,264667,Executing,3
15,264667,Released,2
16,264669,Released,2
17,264671,Released,1
18,264672,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264666,NaN,1.0,1.0
264667,NaN,3.0,2.0
264669,NaN,NaN,2.0
264671,NaN,NaN,1.0
264672,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264620,6.0,9.0,1.0
264654,0.0,21.0,5.0
264655,0.0,10.0,4.0
264661,0.0,16.0,12.0
264663,0.0,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264620,6,9,1
264654,0,21,5
264655,0,10,4
264661,0,16,12
264663,0,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264620,6,9,1
1,264654,0,21,5
2,264655,0,10,4
3,264661,0,16,12
4,264663,0,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264620,6,9,1
1,264654,,21,5
2,264655,,10,4
3,264661,,16,12
4,264663,,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 1:22PM,264672,10,Methapharm-G
1,Jun 13 2023 12:58PM,264671,19,"NAPP Technologies, LLC"
2,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc."
4,Jun 13 2023 12:41PM,264667,10,Alikai Health
9,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd"
11,Jun 13 2023 12:03PM,264664,20,Gutenberg
12,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc."
15,Jun 13 2023 11:54AM,264661,15,"Brookfield Pharmaceuticals, LLC"
43,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd"
69,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2023 1:22PM,264672,10,Methapharm-G,,,1
1,Jun 13 2023 12:58PM,264671,19,"NAPP Technologies, LLC",,,1
2,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc.",,,2
3,Jun 13 2023 12:41PM,264667,10,Alikai Health,,3,2
4,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd",,1,1
5,Jun 13 2023 12:03PM,264664,20,Gutenberg,,,1
6,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc.",,2,1
7,Jun 13 2023 11:54AM,264661,15,"Brookfield Pharmaceuticals, LLC",,16,12
8,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",,21,5
9,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC,,10,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:22PM,264672,10,Methapharm-G,1,,
1,Jun 13 2023 12:58PM,264671,19,"NAPP Technologies, LLC",1,,
2,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc.",2,,
3,Jun 13 2023 12:41PM,264667,10,Alikai Health,2,3,
4,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd",1,1,
5,Jun 13 2023 12:03PM,264664,20,Gutenberg,1,,
6,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc.",1,2,
7,Jun 13 2023 11:54AM,264661,15,"Brookfield Pharmaceuticals, LLC",12,16,
8,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",5,21,
9,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC,4,10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:22PM,264672,10,Methapharm-G,1,,
1,Jun 13 2023 12:58PM,264671,19,"NAPP Technologies, LLC",1,,
2,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc.",2,,
3,Jun 13 2023 12:41PM,264667,10,Alikai Health,2,3,
4,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd",1,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:22PM,264672,10,Methapharm-G,1,NaN,NaN
1,Jun 13 2023 12:58PM,264671,19,"NAPP Technologies, LLC",1,NaN,NaN
2,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc.",2,NaN,NaN
3,Jun 13 2023 12:41PM,264667,10,Alikai Health,2,3.0,NaN
4,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd",1,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:22PM,264672,10,Methapharm-G,1,0.0,0.0
1,Jun 13 2023 12:58PM,264671,19,"NAPP Technologies, LLC",1,0.0,0.0
2,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc.",2,0.0,0.0
3,Jun 13 2023 12:41PM,264667,10,Alikai Health,2,3.0,0.0
4,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd",1,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1058663,9,13.0,0.0
15,529324,13,18.0,0.0
19,1058611,8,31.0,6.0
20,264664,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1058663,9,13.0,0.0
1,15,529324,13,18.0,0.0
2,19,1058611,8,31.0,6.0
3,20,264664,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,9,13.0,0.0
1,15,13,18.0,0.0
2,19,8,31.0,6.0
3,20,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,15,Released,13.0
2,19,Released,8.0
3,20,Released,1.0
4,10,Executing,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,0.0,6.0,0.0
Executing,13.0,18.0,31.0,0.0
Released,9.0,13.0,8.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,0.0,6.0,0.0
1,Executing,13.0,18.0,31.0,0.0
2,Released,9.0,13.0,8.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,0.0,6.0,0.0
1,Executing,13.0,18.0,31.0,0.0
2,Released,9.0,13.0,8.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()